<a href="https://colab.research.google.com/github/ana13S/cs224n_project/blob/main/Data_Processing_Milestone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd

Import datasets

In [53]:
icd = pd.read_csv(r'DIAGNOSES_ICD.csv', dtype="str")
patients = pd.read_csv(r'PATIENTS.csv', dtype="str")
drugs = pd.read_csv(r'PRESCRIPTIONS.csv', dtype="str")

Simulate clinical notes using structured EHR data

In [54]:
for index, row in icd.iterrows():
    subject_id = row["subject_id"]
    notes = ""

    # Find gender
    patient = patients.loc[patients["subject_id"] == subject_id]
    if patient.iloc[0, 2] == "M":
      notes += "Patient gender is male "
    else:
      notes += "Patient gender is female "
    
    # Find dob
    notes += "They were born on " + patient.iloc[0, 3] + " "
    
    # Find prescription drugs
    drug_list = drugs.loc[drugs["subject_id"] == subject_id]
    if drug_list is not None:
      notes += " Prescription drugs are "
    for i, r in drug_list.iterrows():
      notes += r["drug"] + " "
    icd.at[index, "notes"] = notes


icd.to_csv('simulated_notes.csv')



Remove stop words

In [55]:
"""import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
  
for index, row in icd.iterrows():
  word_tokens = word_tokenize(row["notes"])

  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  
  for w in word_tokens:
    if w not in stop_words:
      filtered_sentence.append(w)
  icd.at[index, "notes"] = filtered_sentence

icd.to_csv('preprocessed_simulated_notes.csv')"""

'import nltk\nnltk.download(\'stopwords\')\nnltk.download(\'punkt\')\n\nstop_words = set(stopwords.words(\'english\'))\n  \nfor index, row in icd.iterrows():\n  word_tokens = word_tokenize(row["notes"])\n\n  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]\n  filtered_sentence = []\n  \n  for w in word_tokens:\n    if w not in stop_words:\n      filtered_sentence.append(w)\n  icd.at[index, "notes"] = filtered_sentence\n\nicd.to_csv(\'preprocessed_simulated_notes.csv\')'

Create input for models

In [56]:
from tables.description import IsDescription
input = pd.DataFrame(index=range(len(icd.index)), columns=["notes", "label"])
obesity_icds = ['278', '493', '414', '428', '296', '250', '560', '530', '274', 
                '272', '401', '272', '32723', '7151', '4439', '45981']
# 278 maps to obesity diagnoses in ICD-9
ids = []

for index, row in icd.iterrows():
  if icd.at[index, "subject_id"] not in ids:
    input.at[index, "label"] = 0
    input.at[index, "notes"] = icd.at[index, "notes"]
    ids.append(icd.at[index, "subject_id"])
  
  for code in obesity_icds:
    if icd.at[index, "icd9_code"].startswith(code):
      input.at[index, "label"] = 1
  
input = input.dropna()
print(input)
input.to_csv('model_input.csv', index=False)


                                                  notes label
0     Patient gender is female They were born on 209...     0
21    Patient gender is female They were born on 209...     0
27    Patient gender is female They were born on 203...     0
36    Patient gender is female They were born on 207...     0
50    Patient gender is male They were born on 2114-...     0
...                                                 ...   ...
1691  Patient gender is male They were born on 2057-...     0
1706  Patient gender is male They were born on 1878-...     0
1715  Patient gender is female They were born on 207...     0
1739  Patient gender is male They were born on 2107-...     0
1750  Patient gender is female They were born on 211...     0

[100 rows x 2 columns]
